In [ ]:
import os, json
import re
import pandas as pd
import numpy as np
import glob
import csv
import warnings
from pandas import isnull

# from google.colab import drive
# drive.mount('/content/drive')

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

pattern = r'(_d[1-8][ab])'
mapped_seq_df = pd.DataFrame()
list_of_sequences = []

# PLEASE CHANGE THE PATH ACCORDINGLY
your_path = '/Users/surya/Documents/UoM_Dissertation/'

# /Users/surya/Documents/UoM_Dissertation/artifacts/data/raw/Interaction Data

raw_path = your_path + 'artifacts/data/raw/'
preprocess_path =  your_path + 'artifacts/data/preprocessing/'
prediction_path = your_path + 'artifacts/data/prediction/'
json_path = raw_path + 'Interaction Data/'
json_files = os.listdir(json_path)

unwanted_columns = [ 'screen', 'client', 'episodeCount',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'scroll', 'mouse', 'delta', 'key', 'change', 'select'
       'link', 'class', 'img', 'type', 'inheritedId', 'name', 'textValue', 'text', 'textContent']

necessary_columns = ['event', 'timestampms','node', 'user','id', 'dom', 'flag', 'task', 'merged_event']

total_columns = ['_id', 'event', 'screen', 'client', 'episodeCount', 'timestampms',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'mouse', 'node', 'scroll', 'delta', 'key', 'select', 'change', 'user',
       'type', 'textContent', 'text', 'link', 'inheritedId', 'img', 'class',
       'id', 'textValue', 'name', 'dom', 'flag', 'task', 'merged_event']

id_values = ['toDashboard1', 'toDashboard2', 'toDashboard3', 'toDashboard4', 'toDashboard5',
             'toDashboard6', 'toDashboard7', 'toDashboard8', 'toDashboard9', 'toDashboard10',
             'toDashboard11', 'toDashboard12', 'toDashboard13', 'toDashboard14', 'toDashboard15',
             'toDashboard16']

id_value_until_practice_tutorial = id_values[0]

combined_df = pd.DataFrame()

### function definitions


In [ ]:
### function definitions

def reading_json_files(json_data):
  json_data = re.sub(r'ObjectId\((.+?)\)', r'\1', json_data)
  tempdf = pd.read_json(json_data, lines=True)
  return tempdf


def cleaning_one(tempdf):
  tempdf['user'] = i
  tempdf.drop(['mobileTouch','evID'], axis=1, inplace = True, errors='ignore')
  return tempdf

def splitting_json_column_values(df):
      # Split the key-value pairs in the node column
      df['node'] = df['node'].apply(lambda x: {} if pd.isnull(x) else x)  # Convert NaN values to empty dictionaries
      # Extract the keys from the node dictionary
      keys = set()
      for node_dict in df['node']:
          keys.update(node_dict.keys())
      # Create new columns for each key
      for key in keys:
          df[key] = [node_dict.get(key) if isinstance(node_dict, dict) else None for node_dict in df['node']]

def updating_dom_element(df):
  count = df[df['dom'].notna() & df['dom'].str.contains('\)')].shape[0]
  # Strip the substring before ')' for non-NA values
  df.loc[df['dom'].notna() & df['dom'].str.contains('\)'), 'dom'] = df['dom'].str.split('\)').str[-1]
  df.reset_index(drop=True, inplace=True)

def removing_rows_of_practice_tutorial(df):
  # filling NA values
  df['id'].fillna(-555, inplace=True)
  # ignoring the indices until the practise tutorial
  last_index = df[df['id'] == id_value_until_practice_tutorial].index[-1]
  # Step 2: Remove all rows before the last index
  df = df.loc[last_index:]
  # Reset the index if desired
  df = df.reset_index(drop=True)

def removing_mid_values_between_toDashboardN_ids(df):
  # Remove mid rows between 'toDashboardN' first and last occurrences
  first_occurrences = {}
  last_occurrences = {}
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      first_occurrences[id_value] = df[df['id'] == id_value].index[0]
      last_occurrences[id_value] = df[df['id'] == id_value].index[-1]
  # Step 2: Remove the rows between the first and last occurrence for each value
  for value, first_index in first_occurrences.items():
      last_index = last_occurrences[value]
      df = df.drop(index=range(first_index , last_index))

def dropping_mid_rows_between_toDashboardN_ids(df):
  dropped_rows_count = {id_value: 0 for id_value in id_values}
  for id_value in id_values:
      # Get the indices of the first and last occurrence of the id value
      first_occurrence = df.index[df['id'] == id_value].min()
      last_occurrence = df.index[df['id'] == id_value].max()
      # Determine the indices of the middle rows between the first and last occurrence
      middle_rows_indices = list(range(first_occurrence , last_occurrence))
      dropped_rows_count[id_value] = len(middle_rows_indices)
      df.drop(middle_rows_indices, inplace=True)

  # Reset the index of the dataframe
  df.reset_index(drop=True, inplace=True)

def flagging_toDashboard_rows(df):
  df['flag'] = False
  # Step 2: Set the flag to True for rows matching the 'toDashboard' values
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      df.loc[df['id'] == id_value, 'flag'] = True
      df.reset_index(drop= True, inplace=True)

def merging_events(df):
    df['merged_event'] = df['event']
    # Replace 'mousewheel' with 'scroll'
    df['merged_event'] = df['merged_event'].replace('mousewheel', 'scroll')
    # Merge rows based on the initial conditions
    # df.loc[(df['merged_event'] == 'mouseover') & (df['merged_event'].shift(1) == 'mouseout'), 'merged_event'] = 'hover'
    df['merged_event'] = df['merged_event'].str.replace('mouseover', 'hover').str.replace('mouseout', 'hover')

    df['merged_event'] = df['merged_event'].str.replace('mouseup', 'click').str.replace('mousedown', 'click')
    # df.loc[(df['merged_event'] == 'mouseup') & (df['merged_event'].shift(1) == 'mousedown'), 'merged_event'] = 'click'
    df['merged_event'] = df['merged_event'].str.replace('keyup', 'press').str.replace('keydown', 'press')
    df['merged_event'] = df['merged_event'].str.replace('mobileTouchStart', 'click').str.replace('mobileTouchEnd', 'click')

    # df.loc[(df['merged_event'] == 'keyup') & (df['merged_event'].shift(1) == 'keydown'), 'merged_event'] = 'press'
    df.reset_index(drop=True, inplace=True)

def merging_multiple_events(df):
    df['multi_merged_event'] = df['merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['merged_event'].shift(1) == 'hover') & (df['merged_event'] == 'hover'), 'multi_merged_event'] = 'multihover'
    df.loc[(df['merged_event'].shift(1) == 'click') & (df['merged_event'] == 'click'), 'multi_merged_event'] = 'multiclick'
    df.loc[(df['merged_event'].shift(1) == 'press') & (df['merged_event'] == 'press'), 'multi_merged_event'] = 'multipress'
    df.loc[(df['merged_event'].shift(1) == 'scroll') & (df['merged_event'] == 'scroll'), 'multi_merged_event'] = 'multiscroll'
    df.reset_index(drop=True, inplace=True)

def merging_events_master(df):
    df['event_master'] = df['multi_merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'multihover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'hover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'hover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'multihover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.reset_index(drop=True, inplace=True)

def dropping_multi_hoverings(df):
  df['m_event'] = df['event_master'].ne(df['event_master'].shift()).cumsum()
  df = df[df['event_master'] != 'multihover']  # Remove 'mhover' rows
  df = df.drop_duplicates(subset='m_event', keep='first').drop(columns='m_event')


def initialising_new_column_task(df):
  df['task'] = df['id'].str.extract(pattern)
  df.loc[df['flag'], 'task'] = True
  df.reset_index(drop=True, inplace=True)

def getting_unique_task_values(df_name):
      temp_df = df_name  # Access the DataFrame dynamically using globals()
      # Step 1: Find unique non-NaN values in the 'task' column
      unique_values = temp_df['task'].unique()
      unique_task = next((x for x in unique_values if isinstance(x, str)), None)
      # Step 2: Check if any unique value is a string
      string_pattern = None
      return unique_task

def ignoring_na_dom_values(df):
  df = df[(df['dom'].notna()) | (df['flag'] == True)]
  df.reset_index(drop=True, inplace=True)

def getting_dataframes_task_wise_index(df):
  indices = df.index[df['flag']]
  for i in range(len(indices)-1):
      start = indices[i]
      end = indices[i+1]
      df_name = f"df_db{i+1}"
      globals()[df_name] = df.loc[start:end-1].copy()
  # Create the last DataFrame df_db16 from the last True value to the last row
  start = indices[-1]
  end = df.index[-1]
  df_db16 = df.loc[start:end-1].copy()
  task_wise_split_list = []
  # Step 3: Print the separate DataFrames
  for i in range(1, 16):
      df_name = f"df_db{i}"
      task_wise_split_list.append(globals()[df_name])
  task_wise_split_list.append(df_db16)
  return task_wise_split_list

def dropping_na_values_from_itemset_columns(df_name):
  df_name.loc[df_name['dom'] == '', 'dom'] = np.nan
  df_name.dropna(subset=['dom'], inplace=True)

def dropping_consecutive_rows(df, old_col, new_col):
  consecutive_duplicates = (df['final_event'] == df['final_event'].shift(1))
  # Drop duplicates, keeping only the first occurrence of each consecutive series
  df = df[~consecutive_duplicates]

def dropping_single_transformed_events(df):
  rows_to_remove = df[df['merged_event'] != df['event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)
  df = df.reset_index(drop=True)

def dropping_multi_transformed_events(df):
  rows_to_remove = df[df['multi_merged_event'] != df['merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def dropping_events_master(df):
  rows_to_remove = df[df['event_master'] != df['multi_merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def defining_sequence(series_list):
  # Concatenate all the series into a single series
  concatenated_series = pd.concat(series_list, ignore_index=True)
  # Append -2 to the end of the final series
  concatenated_series = concatenated_series.append(pd.Series([-2]))
  # removing rows with substring of nan values or index True values from 'Task' column
  sequence = concatenated_series[~concatenated_series.apply(lambda x: str(x).endswith(('Tru_e')))]
  return sequence

def replacing_unique_strings_with_numbers(string_1):
    # Split the big string into words and remove duplicates
    words = string_1.split()
    unique_strings = list(set(words))
    # Remove '-1' and '-2' from the list of unique strings
    unique_strings = [s for s in unique_strings if s not in ['-1', '-2']]
    # Create a mapping dictionary between original string values and numbers
    num_mapping = {}
    next_number = 1  # Start assigning numbers from 1000
    for unique_string in unique_strings:
        num_mapping[unique_string] = str(next_number)
        next_number += 1
    # Replace the unique string values in the big string with their corresponding numbers
    for i in range(len(words)):
        if words[i] in num_mapping:
            words[i] = num_mapping[words[i]]
    # Join the words back into a new string
    new_string = ' '.join(words)
    # Create a DataFrame to store the mapping between the original and new string values
    mapping_df = pd.DataFrame(list(num_mapping.items()), columns=['Original_String', 'New_Number'])
    return new_string, mapping_df

def splitting_string_on_negative_two(input_string):
    # Split the input_string whenever "-2" is encountered
    output_string = re.sub(r'-2\s', '-2\n', input_string)
    return output_string

In [ ]:
def defining_itemset(df_name, val):
  df_name['task_val'] =  val
  dropping_na_values_from_itemset_columns(df_name)
  dropping_single_transformed_events(df_name)
  merging_multiple_events(df_name)
  dropping_multi_transformed_events(df_name)
  merging_events_master(df_name)
  dropping_events_master(df_name)

  df_name['itemset'] = df_name['event_master'].astype(str) + "_" + df_name['type'].astype(str)
  # df_name['itemset'] = df_name['merged_event'].astype(str) + "_" + df_name['id'].astype(str) + df_name['dom'].astype(str)
  return df_name['itemset']

def cleaning_three(df_task_list):
  itemset_column = []
  user_sequence = []
  temp_list = []
  for inner_df in df_task_list:
    it = defining_itemset(inner_df, getting_unique_task_values(inner_df))
    itemset_column.append(it.append(pd.Series([-1])))
    temp_list.append(inner_df)
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  # Step 2: Convert the concatenated Series to a single line string
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  # user_sequence.append(defining_sequence(itemset_column))
  return single_line, temp_list

def cleaning_two(data_list):
  df_task_list = []
  one_concatened_dt_list = []
  master_line = ''
  for df in data_list:
      splitting_json_column_values(df)
      updating_dom_element(df)
      removing_rows_of_practice_tutorial(df)
      dropping_mid_rows_between_toDashboardN_ids(df)
      flagging_toDashboard_rows(df)
      initialising_new_column_task(df)
      getting_unique_task_values(df)
      merging_events(df)
      ignoring_na_dom_values(df)
      df_task_list = getting_dataframes_task_wise_index(df)
      line, temp_list = cleaning_three(df_task_list)
      temp_df = pd.concat(temp_list, ignore_index=True)
      one_concatened_dt_list.append(temp_df)
      master_line = master_line + line
  combined_df = pd.concat(one_concatened_dt_list, ignore_index=True)
  return master_line, combined_df


In [ ]:
def combining_similar_types(temp_dataframe):
  temp_dataframe['updated_type'] = temp_dataframe['type']
  type_text = ['B', 'STRONG', 'I', 'P', 'GRAMMARLY-MIRROR', 'GRAMMARLY-EXTENSION', 'CENTER' , 'A']
  type_list = ['LI', 'UL']
  type_heading = ['H2', 'H3', 'H4', 'H5', 'HR']
  type_table = ['TD', 'TH']
  type_button = ['COM-1PASSWORD-BUTTON']
  type_graphic = ['rect', 'tspan', 'path', 'IMG']
  type_basic = ['HTML', 'BODY']
  type_select = ['SELECT', 'OPTION']

  # Use .loc to update 'updated_type' column where 'type' column values match the values_to_update
  temp_dataframe.loc[temp_dataframe['type'].isin(type_text), 'updated_type'] = 'TEXT'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_heading), 'updated_type'] = 'HEADING'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_table), 'updated_type'] = 'TABLE'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_button), 'updated_type'] = 'BUTTON'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_graphic), 'updated_type'] = 'GRAPHIC'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_basic), 'updated_type'] = 'BASIC'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_select), 'updated_type'] = 'SELECTION'
  temp_dataframe.loc[temp_dataframe['type'].isin(type_list), 'updated_type'] = 'LISTING'
  return temp_dataframe

def finding_consecutive_pairs(df):
    # Create a new column 'consecutive_pair' to store consecutive pairs
    df['consecutive_master_pair'] = df['event_master'] + ' -> ' + df['event_master'].shift(-1)
    df['consecutive_itemset_pair'] = df['itemset'] + ' -> ' + df['itemset'].shift(-1)
    # Drop the last row as it will have a NaN value for the 'shift(-1)' operation
    df.drop(df.index[-1], inplace=True)
    # Print the consecutive pairs of values in 'event' column
    merged_consecutive_pairs = df['consecutive_master_pair'].unique()

def removing_consecutive_duplicates(df, column_name, event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    df_two = df[mask].copy()
    while True:
        mask = (df_two[column_name] != event) | (df_two[column_name].shift() != event)
        temp_df = df_two[mask]
        if temp_df.equals(df_two):
            break
        df_two = temp_df.copy()
    return df_two

def replacing_consecutive_events(df, column_name, old_event, new_event):
    mask = (df[column_name] == old_event) & (df[column_name].shift() == old_event)
    df.loc[mask, column_name] = new_event
    return df

def merging_click_sequences_recursive(df, column_name, event, new_event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    df = df[mask]
    if any(df[column_name] == event):
        df = replacing_consecutive_events(df, column_name, event, new_event)
        return merging_click_sequences_recursive(df, column_name, event, new_event)
    else:
        return df

def common_values_across_db(series_list):
  # Initialize a set with the values from the first series
  common_values = set(series_list[0])
  # Iterate through the remaining series and update the common_values set
  for series in series_list[1:]:
      common_values.intersection_update(series)
  # Convert the common_values set back to a list if needed
  common_values_list = list(common_values)
  # Print the common values
  print("Common values:", common_values_list)

def user_mapping_with_timestamp(cleaned_df):
  user_mapping = cleaned_df[['user','sessionstartms']]
  list_time = user_mapping.sessionstartms.unique()
  data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
  user_time = pd.DataFrame(data)
  user_time.to_csv(preprocess_path+'usertimee.csv',index=False)


In [ ]:
data_list = [None] * len(json_files)
i = 1
for file in json_files:
    with open(json_path + file, 'r') as fi:
        json_data = fi.read()
    tempdf = cleaning_one(reading_json_files(json_data))
    data_list[i-1] = tempdf
    i = i+1



In [ ]:
master_df = pd.concat(data_list, ignore_index=True)
master_df.info()

In [ ]:
master_df.url.value_counts()

In [ ]:
master_df.timezoneOffset.value_counts()

In [ ]:
master_df.describe()

In [ ]:
master_df.event.value_counts()

In [ ]:
file_value, cleaned_df = cleaning_two(data_list)
user_mapping_with_timestamp(cleaned_df)
cleaned_df.head()

In [ ]:
cleaned_df.info()

In [ ]:
cleaned_df.type.value_counts()

In [ ]:
final_string, mapping_dataframe = replacing_unique_strings_with_numbers(file_value)
output_string = splitting_string_on_negative_two(final_string)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path +  'one.txt', 'w') as file:
    file.write(output_string)


In [ ]:
finding_consecutive_pairs(cleaned_df)

In [ ]:
temp_dataframe = cleaned_df[['event_master','type', 'itemset', 'id','class','dom','user','task_val', 'consecutive_master_pair', 'consecutive_itemset_pair']]
df_two = removing_consecutive_duplicates(combining_similar_types(temp_dataframe), 'event_master', 'multihover')
df_two.dropna(subset=['task_val'], inplace=True)
finding_consecutive_pairs(df_two)


In [ ]:
df_two

In [ ]:
sorted_df = df_two.sort_values(by=['user','task_val'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

# Create an empty list to store the DataFrames
dataframe_sorted_db_list = []
dataframe_sorted_user_db_list = []

db_user_list = []
for user in unique_user_val:
    user_task_df = sorted_df[sorted_df['user'] == user].copy()
    dataframe_sorted_user_db_list.append(user_task_df)

mapped_dashboard_unique_itemsets = []
user_dashboard_wise_seq = []


In [ ]:
full_seq = ''
for df in dataframe_sorted_user_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  unique_its = []
  for task in unique_task_val:
    task_df = df[df['task_val'] == task].copy()
    task_df['its'] = task_df['event_master'].astype(str) + '__' + task_df['updated_type'].astype(str)
    unique_its.append(task_df['its'])
    itemset_column.append(task_df['its'].append(pd.Series([-1])))
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  full_seq = full_seq + single_line


In [ ]:
final, mapping_user_task_dataframe = replacing_unique_strings_with_numbers(full_seq)
output_full_string = splitting_string_on_negative_two(final)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path + 'all_string.txt', 'w') as file:
    file.write(output_full_string)
mapping_user_task_dataframe

In [ ]:
mapping_user_task_dataframe.to_csv(preprocess_path + "Mapped_values.csv")

In [ ]:
# sorting logic

sorted_df = df_two.sort_values(by=['task_val','user'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

dataframe_sorted_db_list = []
dashboard_wise_seq = []

for task in unique_task_val:
    user_task_df = sorted_df[sorted_df['task_val'] == task].copy()
    dataframe_sorted_db_list.append(user_task_df)

for df in dataframe_sorted_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  for user_no in unique_user_val:
    user_df = df[df['user'] == user_no].copy()
    user_df['its'] = user_df['event_master'].astype(str) + '__' + user_df['updated_type'].astype(str) + '__' + user_df['user'].astype(str)
    itemset_column.append(user_df['its'].append(pd.Series([-1])))

  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"

  str1, map1 = replacing_unique_strings_with_numbers(single_line)
  mapped_dashboard_unique_itemsets.append(map1)
  dashboard_wise_seq.append(str1)


In [ ]:
dataframe_sorted_db_list[0]

In [ ]:
mapped_dashboard_unique_itemsets[0]

In [ ]:
dashboard_wise_seq[0]

In [ ]:
merged_string = ''.join(dashboard_wise_seq)
op_merged = splitting_string_on_negative_two(merged_string)
with open(preprocess_path + 'op_merged.txt', 'w') as fi:
      fi.write(op_merged)


In [ ]:
mapped_dashboard_unique_itemsets

In [ ]:
df_two.to_csv(preprocess_path + "two.csv", index=False)

In [ ]:
finding_consecutive_pairs(df_two)

In [ ]:
df_two.consecutive_master_pair.value_counts()

In [ ]:
df_two.consecutive_itemset_pair.value_counts()

In [ ]:
cleaned_df.event_master.value_counts()

In [ ]:
cleaned_df.info()

In [ ]:
user_mapping = cleaned_df[['user','sessionstartms']]
list_time = user_mapping.sessionstartms.unique()
data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
user_time = pd.DataFrame(data)
user_time.to_csv(preprocess_path + 'usertime.csv',index=False)